In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K

Using TensorFlow backend.


### conv network

In [5]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train[:,:,:,np.newaxis]/255, X_test[:,:,:,np.newaxis]/255

In [43]:
X = X_input = keras.layers.Input(X_train.shape[1:])
X = keras.layers.Conv2D(8, (5,5), padding='same', activation='softmax')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(16, (5,5), padding='same', activation='softmax')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(32, (2,2), padding='valid', activation='softmax')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Conv2D(64, (2,2), padding='valid', activation='softmax')(X)
X = keras.layers.MaxPooling2D()(X)
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(np.max(Y_train)+1, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 28, 28, 8)         208       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 14, 14, 16)        3216      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 6, 6, 32)          2080      
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 3, 3, 32)          0         
__________

In [44]:
M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=64, epochs=50, callbacks=[
    keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1)
])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 316us/step - loss: 1.2379 - acc: 0.5945 - val_loss: 0.3349 - val_acc: 0.9268
Epoch 2/50
60000/60000 [==============================] - 18s 299us/step - loss: 0.2441 - acc: 0.9383 - val_loss: 0.1660 - val_acc: 0.9557
Epoch 3/50
60000/60000 [==============================] - 18s 298us/step - loss: 0.1548 - acc: 0.9575 - val_loss: 0.1277 - val_acc: 0.9637
Epoch 4/50
60000/60000 [==============================] - 18s 299us/step - loss: 0.1211 - acc: 0.9655 - val_loss: 0.1008 - val_acc: 0.9696
Epoch 5/50
60000/60000 [==============================] - 18s 298us/step - loss: 0.0993 - acc: 0.9714 - val_loss: 0.0917 - val_acc: 0.9729
Epoch 6/50
60000/60000 [==============================] - 18s 299us/step - loss: 0.0850 - acc: 0.9749 - val_loss: 0.0791 - val_acc: 0.9753
Epoch 7/50
60000/60000 [==============================] - 18s 301us/step - loss: 0.0751 - acc: 0.9777 - val_loss: 0.

### recurrent network

In [25]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.reuters.load_data()
word_index = keras.datasets.reuters.get_word_index()

In [30]:
embedding_size = max(word_index.values())+1
embedding_size

30980

In [63]:
X = X_input = keras.layers.Input((None,), dtype='int32')
X = keras.layers.Embedding(embedding_size, 32)(X)
X = keras.layers.Bidirectional(keras.layers.LSTM(512, recurrent_activation='softmax'))(X)
X = keras.layers.Dense(np.max(Y_train)+1, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, None, 32)          991360    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 1024)              2232320   
_________________________________________________________________
dense_7 (Dense)              (None, 46)                47150     
Total params: 3,270,830
Trainable params: 3,270,830
Non-trainable params: 0
_________________________________________________________________


In [64]:
def datagen():
    BUCKET_WINDOW = 16
    while True:
        shuffled = list(zip(X_train, Y_train))
        np.random.shuffle(shuffled)
        buckets = {}
        for x,y in shuffled:
            b = int(np.ceil(len(x)/BUCKET_WINDOW))
            if b in buckets:
                bucket = buckets[b]
            else:
                bucket = []
                buckets[b] = bucket
            bucket.append((x+[0]*(BUCKET_WINDOW*b-len(x)),y))
            if len(bucket) >= 64:
                batch, bucket = bucket[:64], bucket[64:]
                buckets[b] = bucket
                yield np.array([row[0] for row in batch]), np.array([row[1] for row in batch])
        for batch in buckets.values():
            if not batch:
                continue
            yield np.array([row[0] for row in batch]), np.array([row[1] for row in batch])
M.fit_generator(datagen(), steps_per_epoch=250, epochs=1)

Epoch 1/1
250/250 [==============================] - 397s 2s/step - loss: 3.3464 - acc: 0.2192
